In [6]:
# 导入库
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# 读取数据
file_path = r"C:\Users\x-mile\Desktop\rm_cw2\Results_21Mar2022.csv"
df = pd.read_csv(file_path, encoding='utf-8')  

# 筛选关键列并处理缺失值
key_columns = ['diet_group', 'sex', 'age_group', 'mean_ghgs', 'mean_land', 'mean_watscar', 'mean_ghgs_ch4', 'mean_watuse']
df = df[key_columns].dropna()

# 标准化数值列
numeric_cols = ['mean_ghgs', 'mean_land', 'mean_watscar', 'mean_ghgs_ch4', 'mean_watuse']
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# 按分组聚合数据
df_agg = df.groupby(['diet_group', 'age_group', 'sex'])[numeric_cols].mean().reset_index()
df_agg.head()  # 查看聚合结果


,diet_group,age_group,sex,mean_ghgs,mean_land,mean_watscar,mean_ghgs_ch4,mean_watuse
0,fish,20-29,female,-0.411154,-0.469730,-0.206988,-0.381262,0.061353
1,fish,20-29,male,-0.166284,-0.367978,0.233715,-0.273870,0.381062
2,fish,30-39,female,-0.398302,-0.466670,-0.116617,-0.389484,0.062586
3,fish,30-39,male,-0.273870,-0.414108,0.109104,-0.366177,0.248245
4,fish,40-49,female,-0.403358,-0.463844,-0.103194,-0.383533,0.027577


In [7]:
# 数值编码映射
# 为分类变量分配数值编码
df_agg['diet_code'] = df_agg['diet_group'].astype('category').cat.codes

# 生成平行坐标图
fig = px.parallel_coordinates(
    df_agg,
    color="diet_code",
    dimensions=numeric_cols,
    color_continuous_scale=px.colors.diverging.Tealrose,
    labels={'diet_code': 'Diet Type'},  # 标签改为英文
    title="Environmental Impact by Diet Type (Standardized)"
)

# 添加分类标签显示
fig.update_layout(
    coloraxis_colorbar=dict(
        title="Diet Type",
        tickvals=df_agg['diet_code'].unique(),
        ticktext=df_agg['diet_group'].unique()
    ),
    width=1000,
    height=600
)
fig.show()

In [8]:
# 雷达图（需添加库导入）
import plotly.graph_objects as go

def plot_radar(df, diet_type):
    df_sub = df[df['diet_group'] == diet_type]
    categories = numeric_cols  # ['mean_ghgs', 'mean_land', ...]
    
    fig = go.Figure()
    for age_group in df_sub['age_group'].unique():
        df_age = df_sub[df_sub['age_group'] == age_group]
        fig.add_trace(go.Scatterpolar(
            r=df_age[numeric_cols].values.flatten().tolist(),
            theta=[col.replace('_', ' ').title() for col in categories], 
            fill='toself',
            name=age_group
        ))
    
    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True)),
        title=f"Environmental Impact by Age Group ({diet_type})",
        width=800,
        height=600
    )
    fig.show()

# 绘制肉食组雷达图
plot_radar(df_agg, 'meat')

In [9]:
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd


# 对每个环境指标进行ANOVA检验
for column in numeric_cols:
    groups = [df_agg[df_agg['diet_group'] == diet][column] for diet in df_agg['diet_group'].unique()]
    f_stat, p_value = stats.f_oneway(*groups)
    print(f"变量 {column}: F值={f_stat:.2f}, p值={p_value:.4f}")
    if p_value < 0.05:
        # 若ANOVA显著，进行事后检验（Tukey HSD）
        tukey = pairwise_tukeyhsd(endog=df_agg[column], groups=df_agg['diet_group'], alpha=0.05)
        print(tukey.summary())
    else:
        print("组间无显著差异\n")

变量 mean_ghgs: F值=1069.15, p值=0.0000
 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
 group1  group2 meandiff p-adj   lower   upper  reject
------------------------------------------------------
   fish    meat   0.7833    0.0  0.6585  0.9081   True
   fish meat100   2.0362    0.0  1.9114  2.1609   True
   fish  meat50   0.2148 0.0001    0.09  0.3396   True
   fish   vegan  -0.8157    0.0 -0.9405 -0.6909   True
   fish  veggie   -0.199 0.0002 -0.3238 -0.0742   True
   meat meat100   1.2529    0.0  1.1281  1.3777   True
   meat  meat50  -0.5685    0.0 -0.6932 -0.4437   True
   meat   vegan   -1.599    0.0 -1.7238 -1.4742   True
   meat  veggie  -0.9823    0.0  -1.107 -0.8575   True
meat100  meat50  -1.8213    0.0 -1.9461 -1.6966   True
meat100   vegan  -2.8519    0.0 -2.9766 -2.7271   True
meat100  veggie  -2.2351    0.0 -2.3599 -2.1104   True
 meat50   vegan  -1.0305    0.0 -1.1553 -0.9057   True
 meat50  veggie  -0.4138    0.0 -0.5386  -0.289   True
  vegan  veggie   0.6167    0